# Difference in nonconformity and anomaly scores
This script evaluates anomaly detection algorithms based on their capability to differentiate between normal and anomalous segments in a time series. For this purpose, the nonconformity and anomaly scores are analyzed for an anomalous segment, compared to the average score level directly before the anomaly. Anomalies can be labeled anomalies found in benchmark datasets or artificially introduced anomalies.

In [53]:
import os
import re
import json
import numpy as np
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
from matplotlib.pyplot import cm

from models.gnn_ensembles import EnsembleGNNWrapper
from training_set_update.slidingWindow import sliding_window as sw_helper
from nonconformity_scores.nonconformity_wrapper import calc_nonconformity_scores

## Paths

In [64]:
buffer_preceding_interval_anomaly = 100
preceding_anomaly_interval_length = 50
preceding_offset = preceding_anomaly_interval_length + buffer_preceding_interval_anomaly

In [60]:
data_representation_length, dataset_category = 100, 'multivariate'
collection_id = 'Daphnet'
model_id = 'ensemble_gnn'
ls_id, anomaly_score_id = 'ares_al_ks', 'anomaly_likelihood'
run_date_id = '20240304_112528'
output_folder_path = f'../out/{collection_id}'
data_folder_base_path = f'../data/{dataset_category}/{collection_id}'
dataset_ids = sorted([x for x in os.listdir(output_folder_path) if not x.startswith('.')])

## Analyze nonconformity scores after finetuning

In [61]:
def calculate_anomaly_sequences(target):
    anomaly_sequences = []
    anomaly_indices = np.unique(np.where(target == 1)[0])
    change_ind = np.where(np.diff(anomaly_indices) != 1)[0] + 1
    if len(change_ind) != 0:
        sequences = np.split(anomaly_indices, change_ind)
    else:
        sequences = [anomaly_indices]
    for sequence in sequences:
        if len(sequence) != 0:
            anomaly_sequences.append([np.min(sequence), np.max(sequence)])
    return anomaly_sequences

In [62]:
def overlap(start1, end1, start2, end2):
    """Does the range (start1, end1) overlap with (start2, end2)?"""
    return not (end1 < start2 or end2 < start1)

In [66]:
results_total = {}
for dataset_id in dataset_ids:
    print(f"Now at dataset {dataset_id}")
    results = {}
    test_data = pd.read_csv(f'{data_folder_base_path}/{dataset_id}.test.csv')
    labels = test_data['is_anomaly'].to_numpy()
    if len(np.unique(labels)) == 1:
        continue
    true_anomaly_sequences = calculate_anomaly_sequences(labels)
    artificial_anomalies_path = f'{output_folder_path}/{dataset_id}/artificial_anomalies/{run_date_id}'
    artificial_anomaly_sequences = [[int(x.group()) for x in re.finditer(r'\d+', fn)] for fn in os.listdir(artificial_anomalies_path) if not fn.startswith('.')]
    all_anomaly_sequences = artificial_anomaly_sequences + true_anomaly_sequences
    approaches_paths = [x for x in os.listdir(f'{output_folder_path}/{dataset_id}') if not x.startswith('.') and not x in ['initial_weights', 'artificial_anomalies']]
    for approach_path in approaches_paths:
        model_id, learning_strategy_id, anomaly_score_id = approach_path.split('-')
        if model_id not in results.keys():
            results[model_id] = {}
        if learning_strategy_id not in results[model_id].keys():
            results[model_id][learning_strategy_id] = {}
        if anomaly_score_id not in results[model_id][learning_strategy_id].keys():
            results[model_id][learning_strategy_id][anomaly_score_id] = []
        score_path = f'{output_folder_path}/{dataset_id}/{approach_path}/{run_date_id}'
        anomaly_scores = pd.read_csv(f'{score_path}/anomaly_scores.csv').to_numpy()
        nonconformity_scores = pd.read_csv(f'{score_path}/nonconformity_scores.csv').to_numpy()
        offset = int(anomaly_scores[0, 0])

        # compare anomaly maximum nc/anomaly score to previous average (check that this sequence is not actually another previous anomaly) and save in results dict
        for seq in all_anomaly_sequences:
            start, end = seq[0] - offset, seq[1] + 1 - offset
            if start - preceding_offset < 0:
                continue
            max_anomaly_score = anomaly_scores[start:end, 1].max()
            max_nonconformity_score = nonconformity_scores[start:end, 1].max()
            if not any([overlap(seq[0] - preceding_offset, seq[0] - 1, seq2[0], seq2[1]) for seq2 in all_anomaly_sequences]):
                # print(f'Anomaly sequence: {seq[0]} - {seq[1]}')
                preceding_anomaly_mean = np.mean(anomaly_scores[start - preceding_offset : start - buffer_preceding_interval_anomaly, 1])
                preceding_nonconformity_mean = np.mean(nonconformity_scores[start - preceding_offset : start - buffer_preceding_interval_anomaly, 1])
            else:
                # print(f'Skipping anomaly sequence: {seq[0]} - {seq[1]}')
                continue
            results[model_id][learning_strategy_id][anomaly_score_id].append({
                "anomaly_sequence": str(seq),
                "is_artificial": seq in artificial_anomaly_sequences,
                "preceding_anomaly_mean": preceding_anomaly_mean,
                "max_anomaly_score": max_anomaly_score,
                "preceding_nonconformity_mean": preceding_nonconformity_mean,
                "max_nonconformity_score": max_nonconformity_score,
            })
        
    results_total[dataset_id] = results

Now at dataset S01R01E0
Now at dataset S01R01E1
Now at dataset S01R02E0
Now at dataset S02R01E0
Now at dataset S02R02E0
Now at dataset S03R01E0
Now at dataset S03R01E1
Now at dataset S03R02E0
Now at dataset S03R03E0
Now at dataset S03R03E1
Now at dataset S03R03E2
Now at dataset S03R03E3
Now at dataset S03R03E4
Now at dataset S04R01E0
Now at dataset S04R01E1
Now at dataset S05R01E0
Now at dataset S05R01E1
Now at dataset S05R01E2
Now at dataset S05R01E3
Now at dataset S05R02E0
Now at dataset S05R02E1
Now at dataset S06R01E0
Now at dataset S06R01E1
Now at dataset S06R01E2
Now at dataset S06R02E0
Now at dataset S06R02E1
Now at dataset S07R01E0
Now at dataset S07R02E0
Now at dataset S08R01E0
Now at dataset S08R01E1
Now at dataset S08R01E2
Now at dataset S08R01E3
Now at dataset S09R01E0
Now at dataset S09R01E1
Now at dataset S09R01E2
Now at dataset S09R01E3
Now at dataset S09R01E4
Now at dataset S10R01E0
Now at dataset S10R01E1


In [67]:
with open(f'{output_folder_path}/results_difference_nc_anomaly_scores.json', 'w') as file:
    json.dump(results_total, file)

## Post Processing
Calculate absolute and relative difference averaged over all files

In [68]:
template = {'as_abs': [], 'as_rel': [], 'nc_abs': [], 'nc_rel': []}
template_outer = {
    'artificial_anomalies': deepcopy(template),
    'real_anomalies': deepcopy(template),
    'all_anomalies': deepcopy(template)
}
results_post_processing = {
    'models': {},
    'learning_strategies': {},
    'models_learning_strategies': {}
}
for dataset_id in results_total.keys():
    approaches_paths = [x for x in os.listdir(f'{output_folder_path}/{dataset_id}') if not x.startswith('.') and not x in ['initial_weights', 'artificial_anomalies']]
    for approach_path in approaches_paths:
        model_id, learning_strategy_id, anomaly_score_id = approach_path.split('-')
        model_ls_id = f'{model_id}-{learning_strategy_id}'
        if anomaly_score_id == 'confidence_levels':
            continue
        if model_id not in results_post_processing['models'].keys():
            results_post_processing['models'][model_id] = deepcopy(template_outer)
        if learning_strategy_id not in results_post_processing['learning_strategies'].keys():
            results_post_processing['learning_strategies'][learning_strategy_id] = deepcopy(template_outer)
        if model_ls_id not in results_post_processing['models_learning_strategies'].keys():
            results_post_processing['models_learning_strategies'][model_ls_id] = deepcopy(template_outer)
        for dict_key, obj_id in [('models', model_id), ('learning_strategies', learning_strategy_id), ('models_learning_strategies', model_ls_id)]:
            for anomaly_entry in results_total[dataset_id][model_id][learning_strategy_id][anomaly_score_id]:
                if anomaly_entry['is_artificial']:
                    anomaly_categories = ['artificial_anomalies', 'all_anomalies']
                else:
                    anomaly_categories = ['real_anomalies', 'all_anomalies']
                for anomaly_category in anomaly_categories:
                    results_post_processing[dict_key][obj_id][anomaly_category]['as_abs'].append(anomaly_entry['max_anomaly_score'] - anomaly_entry['preceding_anomaly_mean'])
                    results_post_processing[dict_key][obj_id][anomaly_category]['as_rel'].append(anomaly_entry['max_anomaly_score'] / anomaly_entry['preceding_anomaly_mean'] - 1.0)
                    results_post_processing[dict_key][obj_id][anomaly_category]['nc_abs'].append(anomaly_entry['max_nonconformity_score'] - anomaly_entry['preceding_nonconformity_mean'])
                    results_post_processing[dict_key][obj_id][anomaly_category]['nc_rel'].append(anomaly_entry['max_nonconformity_score'] / anomaly_entry['preceding_nonconformity_mean'] - 1.0)
                
       
# Average across categories
for dict_key in results_post_processing.keys():
    for obj_key in results_post_processing[dict_key].keys():
        for anomaly_category in template_outer.keys():
            for score_key in template.keys():
                results_post_processing[dict_key][obj_key][anomaly_category][score_key] = sum(results_post_processing[dict_key][obj_key][anomaly_category][score_key]) / max(1, len(results_post_processing[dict_key][obj_key][anomaly_category][score_key]))

In [69]:
with open(f'{output_folder_path}/results_post_processing_difference_nc_anomaly_scores.json', 'w') as file:
    json.dump(results_post_processing, file)